In [ ]:
%%capture 
%%shell
wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib
sudo ./configure
sudo make
sudo make install
pip install TA-Lib  python-binance  unsync icecream #pystan==2.19.1.1 prophet 

In [ ]:
def today():
  from datetime import datetime as dt
  x = dt.now()
  x = x.replace(second=0,microsecond=0)
  return x

def deltaTime(min=1,sec=0):
  from datetime import timedelta as td
  return td(minutes=min,seconds=sec)

def to_date(column):
  return pd.to_datetime(column,unit='ms')

def futureTime(date=today(),min=1):
 return date+deltaTime(min)
 
pluck = lambda dict, *args: (dict[arg] for arg in args)

#Assist Pandas
filter_df = lambda df,cln,val: df.loc[df[cln]==val]
pluck_row = lambda row,*clns: (row[cln].iloc[0] for cln in clns)
drop_cols = lambda df,*clns:df.drop(list(clns),axis=True,inplace=True) 
num_col = lambda col: pd.to_numeric(col) 

symboler = lambda crypto: f'{crypto}USDT' if crypto and len(crypto)<6 else crypto

In [ ]:
#https://docs.python.org/3/library/logging.config.html
x = '''
[loggers]
keys=root,binanceLogger

[handlers]
keys=consoleHandler,fileHandler,fileHandler2

[formatters]
keys=sampleFormatter

[logger_root]
level=INFO
handlers=consoleHandler,fileHandler

[logger_binanceLogger]
level=INFO
handlers=fileHandler2
qualname=binanceLogger
propagate=0

[handler_consoleHandler]
class=StreamHandler
level=INFO
formatter=sampleFormatter
args=(sys.stdout,)

[handler_fileHandler]
class=FileHandler
level=DEBUG
formatter=sampleFormatter
args=('bot.log','a')

[handler_fileHandler2]
class=FileHandler
level=INFO
formatter=sampleFormatter
args=('bince.log','a')

[formatter_sampleFormatter]
format=%(asctime)s - %(levelname)s - %(message)s
datefmt='%H:%M:%S'
'''
with open('file.conf','w') as f1: f1.write(x)

In [ ]:
from icecream import ic #use install to install in all modules
import logging as log
import time
import logging.config as log_config
from queue import Queue
from dask import delayed,compute
import math

# log_config.fileConfig(fname='file.conf', disable_existing_loggers=False) 

def info(s):
  print(f'{s}')

def prefix_time():
  return f'{time.time()}'

# log.basicConfig(filename='app.log', filemode='w', format='%(asctime)s %(process)d %(name)s - %(levelname)s - %(message)s ', datefmt='%d-%b-%y %H:%M:%S',level=log.INFO)
# log.basicConfig(level=log.INFO)
ic.configureOutput(prefix=prefix_time,outputFunction=info,includeContext=False)
log_config.fileConfig(fname='file.conf', disable_existing_loggers=False) #Auto calles basicConfig and mods 
bLog = log.getLogger('binanceLogger')

In [ ]:
from threading import Thread, enumerate as threads, current_thread,Lock
from multiprocessing import Process

def keepAlive(fc):
    def inner(*args):
        t = Thread(name=fc.__name__, target=fc, args=args, daemon=1)
        t.start()
    return inner

def undead(fc):
    def inner(*args):
        p = Process(name=fc.__name__, target=fc, args=args, daemon=True)
        p.start()
    return inner


In [ ]:
class Position():
  q,c = 0,0 # c is entry price, qis quantity
  flag = True
  def __init__(self,mode,crypto,profit,hedge=True,open_args={},close_args={},trend=math.inf): #basically take 0-long
    self.pos = 'SHORT' if mode=='S' else 'LONG'
    self.profit = profit
    self.crypto = crypto
    self.PRECISION = precision(crypto)
    self.hedge = hedge
    self.open_args= open_args
    self.close_args = close_args
    self.trend = 1
    self.refresh()
  def __call__(self):
    print(self.status())
  def status(self):
    return self.q,self.c,self.pos
  def __repr__(self):
    return f'{self.status()}'
  @property
  def cost(self):
    return self.c*self.q if self.q >0 else 0
  def pnl(self,p,q=0):
    q = q if q else self.q 
    net = (p-self.c)*q
    if self.pos=='SHORT': return -net
    return net
  @delayed
  def open(self,q,p=''):
    if not (self.hedge and self.flag): return f"{self.crypto} {self.pos} disabled"
    d = self.open_args
    d['quantity'] = q
    paras = ['price','profit','loss']
    for i in paras:
      if i in d: d[i]=d[i] or p
    x = order(self.crypto,'OPEN',self.pos,**d) 
    self.refresh() 
    return x
  @delayed
  def close(self,p='',q='',**args):
    d = self.close_args
    d.update(args)
    d['quantity'] = q or self.q
    if p:
      paras = ['price','profit','loss']
      for i in paras:
        if i in d: d[i]=p
    x = order(self.crypto,'CLOSE',self.pos,**d)
    self.refresh()
    return x
  @keepAlive
  def refresh(self):
    data = positions(self.crypto,self.pos)
    self.q,self.c = pluck_row(data,'positionAmt','entryPrice')
  def sell_signal(self,p,q=''):
    return self.pnl(p,q)>=self.profit
  @property
  def liqPrice(self):
     data = positions(self.crypto,self.pos)
     liq = pluck_row(data,'liquidationPrice')
     return liq 
  @property
  def max_profit(self):
      if self.q>0:
        p = (self.profit*2)/self.q 
        p = self.c-p if self.pos=='SHORT' else self.c+p
      else: return 0
      return p

RSI Strategy

In [ ]:
import numpy as np
import talib
import time


rsi_q = Queue(100)
macd_q = Queue(100)

def RSIStrategy(price,period=14): 

  if CLOSES.size <=period: return 

  rsi = talib.RSI(CLOSES,period)
  cur_rsi = rsi[-1]

  rsi_q.put((price,cur_rsi))

@keepAlive
def rsing():
  under,over = 20,80
  signal = -1
  while 1:
    price,cur_rsi = rsi_q.get()
    if cur_rsi <= under and signal !=0: 
      ic("OverSold")
      buys.put(price)
      signal = 0
    elif cur_rsi >= over and signal !=1:
      ic("OverBought")
      sells.put(price)
      signal = 1

def cur_macd(prices,fast,slow,signal): #talib is faster ...
  macds = talib.MACD(prices, fastperiod=fast, slowperiod=slow, signalperiod=signal) #macd,signal,histogram
  m,s,h = [macds[i][-1] for i in range(3)]
  return m,s 



def MACDStrategy(price,fast=12,slow=26,signal=9): #CLOSES must be updated..
  macd,signal = cur_macd(CLOSES,fast,slow,signal)
  macd_q.put((price,macd,signal))
  
@keepAlive
def macding():    
  signal = 0 # change quantity according to signal averaging method but avoid for now
  while 1:
      data = macd_q.get()
      ic(f"MACD {data}")
      p,m,s = data #pluck(data,'price','macd','signal')
      if m > s:
          if signal != 1:
              buys.put(p)
              signal=1
      elif m < s:
          if signal != -1:
              sells.put(p)
              signal = -1


In [ ]:
buys = Queue(100)
sells = Queue(100)
closes = Queue(100)
opens = Queue(100)


@keepAlive
def buying(q,long,short):
  while 1:
    p = buys.get()
    if short.sell_signal(p): closes.put([short.close(p),short])
        
    if long.flag: 
      if short.q>0 and short.c<p: continue
      opens.put([long.open(q,p),long]) 

@keepAlive
def selling(q,long,short):
  while 1:
    p = sells.get()
    if long.sell_signal(p): 
      closes.put([long.close(p),long])

    if short.flag:
      if long.q>0 and long.c>p: continue
      opens.put([short.open(q,p),short]) 

In [ ]:
#Instead of pos.hedge : - flag - for global sell and pos.flag to temp lock
@keepAlive
def opening(): 
  while 1:
      n,pos = opens.get()
      try:  
        x = n.compute()
          # if pos.max_profit>0: closes.put([pos.close(price=pos.max_profit),pos])   #can cause losses in case...
        bLog.info(x)
      except Exception as e:
        if hasattr(e,'code'):
          bLog.error(f"@open {e.code} {e.message}")
          over_flow = [-2027,-2019,-4164] #max lev,margin insufficient,order notional
          if e.code in over_flow and pos.q>0: pos.flag  = False
        else:
          log.exception('@open')

@keepAlive
def closing():
    while 1:
        n,pos = closes.get()
        try:  
          x = n.compute()
          bLog.info(x)
          transfer_to('SPOT',amt=.6*pos.profit) #babylonian
        except Exception as e:
          # bLog.exception("@close")
          if hasattr(e,'code'):
            bLog.error(f"@close {e.code} {e.message}")
            if e.code==-4045: cancel_order(pos.crypto) #max stop Order  #-2021 would immediately trigger
          else:
            log.exception('@close')
        else:
          ic('Sell Successfull') 
          pos.flag = True 

Binance Trade

In [ ]:
from binance import Client,AsyncClient,BinanceSocketManager
from unsync import unsync

def gen_client(testnet=True, asyn=False):
  napi_key = "e6geB7TFiaM4FTACiiP2Slk9QFQyYuCt37KH7ht5nwrYcJCvdkloyHAzh4ImtSQh"
  napi_secret = "dC25QStiRNVlGrHJk8ajK9fQhmUCCDfGwSjTvqCFP0Fsdyw3nKtmkZjgKrmmwGIZ2"

  api_key="9Cc4lp3DlqB9sh7Si2a63sktrhAUiPmtRTMTh22v2OOaNweYSXcHoKzEcPtlLX5i"
  api_secret="xbw7wRuGvNjDXFS1rM5BTkoRToJfWW8StEUvO0tct9Ibwz3dj9vXThQTzF1iIEeE"

  if testnet:
    api_key="17decb9a46f67558c5088856ee8c3fa28c20c72de37bb8cffc9924a2a9cc0a12"
    api_secret='7e27cf5baa00ab22734a4781d5050d20be40ab0c2289567769eb2a789d3a294e'


  if asyn:client =  AsyncClient(api_key, api_secret,testnet=testnet)#,tld='us')
  else: client =  Client(api_key, api_secret,testnet=testnet)#,tld='us')
  
  return client

@unsync
async def socket_manager(client=''):
    c1 = await AsyncClient.create() if not client else client
    socket = BinanceSocketManager(c1)
    return socket

#  dir(client)

In [ ]:
symboler = lambda crypto: f'{crypto}USDT' if crypto and len(crypto)<6 else crypto
get_price = lambda crypto:float(client.futures_symbol_ticker(symbol=symboler(crypto))['price'])
market_price = lambda crypto : float(client.get_symbol_ticker(symbol=symboler(crypto))['price'])
min_q= lambda crypto:precision(crypto)[-1]
_min_q = lambda crypto,notional,qP:round(notional /get_price(crypto),qP) or pow(10,-qP) #.001

def exchange(crypto=''):
  info = client.futures_exchange_info()
  df = pd.DataFrame(info['symbols'])
  df = filter_df(df,'contractType','PERPETUAL')
  df = filter_df(df,'quoteAsset','USDT')
  drop_cols(df,'pair','deliveryDate','onboardDate','contractType','status','quoteAsset','marginAsset','timeInForce','orderTypes','settlePlan','underlyingSubType','underlyingType','liquidationFee')
  x = df.loc[df['symbol']==symboler(crypto)] if crypto else df
  return x
  
def precision(crypto):
  e = exchange(crypto)
  res = [int(i) for i in e.iloc[0,4:8].to_list()] #pricePrecision	quantityPrecision	baseAssetPrecision	quotePrecision

  filter = pd.DataFrame(e['filters'].iloc[0])
  num_cols = [i for i in filter.columns if i != 'filterType']
  for i in num_cols:filter[i]=num_col(filter[i])
  filter['notional'] = filter['notional'].apply(lambda x: _min_q(crypto,x,res[1]))
  d = {'PRICE_FILTER':['minPrice','maxPrice'],'MIN_NOTIONAL':['notional']}
  for name,filters in d.items():
    f_df = filter_df(filter,'filterType',name)
    res+=pluck_row(f_df,*filters)
  return res 

def positions(crypto='',pos=''):
  data = client.futures_position_information()
  df = pd.DataFrame(data)
  cols = ['positionAmt','entryPrice','markPrice','unRealizedProfit','liquidationPrice','leverage','isolatedWallet']
  drop_cols(df,'updateTime')
  for col in cols: df[col] = num_col(df[col])
  df = filter_df(df,'symbol',symboler(crypto)) if crypto else  df.loc[df['positionAmt']>0]
  df = filter_df(df,'positionSide',pos) if pos else df
  df['positionAmt'] = df['positionAmt'].apply(abs) #SHORT side amounts are neg
  return df

def income(profit=True, pnl=True):
  df = pd.DataFrame(client.futures_income_history())
  df['income'] = num_col(df['income'])
  df['date'] = to_date(df['time'])
  drop_cols(df,'time','tranId','info')
  df = df.sort_values(by=['date'],ascending=False)
  df =  df.loc[df['incomeType']=='REALIZED_PNL'] if pnl else df
  df = df.loc[df['income']>0] if profit else df
  # df = df.loc[df['incomeType']!='TRANSFER']
  df['profit'] = df['income'].sum()
  return df

def wallet():
  df = pd.DataFrame(client.futures_account_balance())
  df['balance'] = num_col(df['balance'])
  # df = df.loc[df['balance']>0]
  # df = df.reset_index(drop=True)
  df.drop(['accountAlias'],axis=1,inplace=True)
  return df

def wallet_spot(crypto='USDT'):
  df = pd.DataFrame(client.get_asset_balance(asset=crypto),index=[0])
  for i in ['free','locked']: df[i] = num_col(df[i])
  return df

def balance(crypto='USDT',type='FUT'):
  b=0
  if type=='FUT':
    df = wallet()
    b = df.loc[df['asset']==crypto]['balance'].iloc[0]
  elif type =='SPOT':
    b = wallet_spot(crypto)['free'].iloc[0]
  return b


def transfer_to(dest='SPOT',amt=0,crypto='USDT'):
  if dest=='SPOT': 
    type =2
    src = 'FUT'
  else: 
    type =1
    src = "SPOT"
  amt = amt or balance(crypto,type=src)
  if amt>0:
    return client.futures_account_transfer(asset=crypto,amount=amt,type=type)
  bLog.error(f"Transfer failed {src}->{dest} {amt}")
  return -1


def open_orders(crypto=''):
  data = client.futures_get_open_orders(symbol=symboler(crypto))
  if len(data)==0: return -1
  df = pd.DataFrame(data)
  df['date'] = to_date(df['time'])
  # df.drop(['updateTime','time','cumQuote','price','avgPrice','origQty','executedQty','status'],axis=1,inplace=True)
  return df

def cancel_order(crypt,id=''):
  d = {'orderId':id,'symbol':symboler(crypt)}
  return client.futures_cancel_all_open_orders(**d) #{'code': 200, 'msg': 'The operation of cancel all open order is done.'}

def order(crypto,side,mode,quantity='',price='',profit='',loss='',trail='',PRECISION=''): 
  p,q,bP,qP,minP,maxP,minQ = PRECISION or precision(crypto)

  if quantity: quantity = round(max(quantity,minQ),q)

  d = {'type':'MARKET','symbol':symboler(crypto), "quantity":quantity,'positionSide':mode} 
  
  if price: 
    price = round(price,p)
    price = min(maxP,price)
    price = max(minP,price)
    d['type'] = 'LIMIT'
    d['price'] = price
    d['timeInForce']='GTC' 

  if profit:
        d['type']='TAKE_PROFIT' if price else 'TAKE_PROFIT_MARKET'
        d['stopPrice'] = round(profit,p)
  elif loss:
    d['type']='STOP' if price else 'STOP_MARKET'  
    d['stopPrice'] = round(loss,p)
  elif trail:
        d['type'] = 'TRAILING_STOP_MARKET'
        d['callbackRate'] = trail
        d['activationPrice'] = price 

  if side=='OPEN' :
    d['side']  = 'BUY' if mode == 'LONG' else 'SELL'
  else :
     d['side']  = 'SELL' if mode == 'LONG' else 'BUY'
     if quantity=='' and (profit or loss): 
        d['closePosition'] = True      
        del d['quantity']  

  log.info(d)

  x=client.futures_create_order(**d)
  df = pd.DataFrame(x,index=[0]).rename(columns={"":"position"})
  drop_cols(df,*['status','clientOrderId','avgPrice','executedQty','cumQty','cumQuote','timeInForce','reduceOnly','workingType','priceProtect','origType','updateTime'])
  ncol = ['activatePrice','origQty','price','priceRate','stopPrice']
  for i in ncol: 
    if i in df.columns:df[i] = num_col(df[i])
  x = df.iloc[0].to_dict()
  return x
#https://dev.binance.vision/t/what-does-the-percent-price-filter-mean/134

In [ ]:
def status():
  status_code = client.get_system_status()['status']
  if(status_code!=0):
     print("Exiting")
     sys.exit(1)

def permissions():
  return client.get_account_api_permissions()

def position(hedge=True):
  client.futures_change_position_mode(dualSidePosition = hedge)
  return client.futures_get_position_mode()

def my_trades():
  df = pd.DataFrame(client.futures_account_trades())
  df['time'] = to_date(df['time'])
  return df

def history(crypto,interval='1m',start_str='1 day ago'):
  data = client.futures_historical_klines(
    symbol=symboler(crypto),
    interval=interval,  # can play with this e.g. '1h', '4h', '1w', etc.
    start_str = start_str,
    # end_str='2021-06-30'
)
  df = pd.DataFrame(data)
  # crop unnecessary columns
  df = df.iloc[:, :6]
  # ascribe names to columns
  df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
  # convert timestamp to date format and ensure ohlcv are all numeric
  df['date'] = to_date(df['date'])
  for col in df.columns[1:]:
      df[col] = pd.to_numeric(df[col]) 
  return df

def make_history(df): #multable so no issues
  df['cap']= df['high'].mean()  #cap can be increasing sequence...
  df['floor'] = df['low'].min()
  df.drop(['volume','open','high','low','volume'],axis=1,inplace=True)     #only low
  df.columns = ['ds','y','cap','floor']
  return df

def leverage_bracket(crypto):
  symbol = symboler(crypto)
  data = client.futures_leverage_bracket(symbol=symbol)[0]
  df = pd.DataFrame(data['brackets'])
  return df

def max_leverage(crypto='BTC'):
  df = leverage_bracket(crypto)
  return df['initialLeverage'].max()

def leverage(crypto,lev=''):
  lev = lev if lev else max_leverage(crypto)
  return client.futures_change_leverage(symbol=symboler(crypto), leverage=lev)

def order_book(crypto='BTC'):
  data =client.futures_order_book(symbol=symboler(crypto))
  return data

def recent_trades(crypto):#aggrefate
  s  = symboler(crypto)
  data = client.futures_aggregate_trades(symbol=s,limit=1000)
  df = pd.DataFrame(data)
  df['price'] = num_col(df['p'])
  return df


Socket

In [ ]:
import asyncio
from binance import AsyncClient,Client, BinanceSocketManager
from unsync import unsync
import math 

ac = gen_client(True,True)

socket = socket_manager(ac).result()

async def stream_client():
    c1 = await AsyncClient.create()
    SC = BinanceSocketManager(c1)
    return SC

async def trade_socket(crypto='BTC',SC=socket,type='F'):
    crypto= f'{crypto}USDT'if len(crypto) <6 else crypto
    
    if type=='F':
      conn = SC.aggtrade_futures_socket(crypto)
    elif type =='S':
      conn = SC.trade_socket(crypto)
    elif type=='SA' :
      conn = SC.aggtrade_socket(crypto)
    return conn

async def kline_socket(crypto='BTC',interval='1m',SC=socket,type='F'):
  crypto= f"{crypto}USDT'if len(crypto) <6 else crypto"
  d = {'symbol':crypto,'interval':interval}
  if type=='S': return SC.kline_socket(**d) 
  return SC.kline_futures_socket(**d)

async def market_socket(crypto='BTC',SC=socket):  
  crypto=symboler(crypto)
  return SC.symbol_mark_price_socket(crypto) #Futures Socket

def handle_message(msg):
        ic(msg)
        # print(f"Time: {msg['T']} - Symbol: {msg['s']} - Price: {msg['p']} - Quantity: { msg['q']} -Market: {msg['m']}")

@unsync
async def socket_handler(conn=trade_socket(),msg_handler=handle_message,y=17):
    try:  
      conn = await conn
    except:
      log.debug("Already awaited")
    async with conn as SM:
        i=0
        while i<y:
          try:
            res = await SM.recv()
          except:
            log.debug('Already SM')
            # print(res)
          msg_handler(res)
          i+=1
            # asyncio.sleep(1)
        # await ts.__aenter__().
        # await ts.__aexit__(None, None, None)

def handle_market_socket(msg):
  d = msg['data']
  S,T,p = pluck(d,'s','E','p')
  p = float(p)
  global CLOSES
  CLOSES = np.append(CLOSES,p) 
  # RSIStrategy(p,7)
  # rsing()
  macding()
  MACDStrategy(p,12,26,9)


import math
@unsync
async def main():
    # then start receiving messages
  t = math.inf
  max = market_socket(CRYPTO)
  await socket_handler(max,handle_market_socket,t)
# https://livedataframe.com/live-cryptocurrency-data-python-tutorial/
#https://github.com/binance/binance-spot-api-docs/blob/master/web-socket-streams.md#trade-streams

In [ ]:
client = gen_client(testnet=1)


In [ ]:
import pandas as pd
# CLOSES = np.array([])
crypts = {'BTC':.01,'HOT':2000}

BALANCE = balance('USDT')

CRYPTO = 'DENT'
PRECISION = precision(CRYPTO)
# AMOUNT =crypts[CRYPTO]
AMOUNT = min_q(CRYPTO)
PROFIT = BALANCE*.01 #3%
LONG = Position('L',CRYPTO,PROFIT) 
SHORT =Position('S',CRYPTO,PROFIT)

r_trades = recent_trades(CRYPTO)
CLOSES = r_trades['price'].to_numpy()
CLOSES_1m = history(CRYPTO,'1m','15 min ago')
CLOSES_3m = history(CRYPTO,'1m','1 hr ago')
CLOSES_1h = history(CRYPTO,'1h','12  hr ago')
CLOSES_1d = history(CRYPTO,'1d','12 day ago')

In [ ]:
df =history('BTC')
df

1671110939.139688f"MACD {data}": 'MACD (0.00069874, 1.7921261155997605e-07, 1.6038303895376634e-07)'
1671110940.2325578f"MACD {data}": 'MACD (0.00069874, 1.7354114298232578e-07, 1.6301465975947824e-07)'


,date,open,high,low,close,volume
0,2022-12-14 13:29:00,17928.5,17938.0,17901.9,17914.9,461.133
1,2022-12-14 13:30:00,17914.9,17938.0,17905.0,17912.8,217.104
2,2022-12-14 13:31:00,17912.8,17912.8,17890.6,17900.1,196.695
3,2022-12-14 13:32:00,17900.1,17900.1,17805.9,17890.3,300.141
4,2022-12-14 13:33:00,17895.0,17900.1,17879.6,17886.5,339.732
...,...,...,...,...,...,...
1435,2022-12-15 13:24:00,17560.0,17562.3,17547.5,17553.2,51.562
1436,2022-12-15 13:25:00,17553.2,17563.1,17542.0,17542.0,520.463
1437,2022-12-15 13:26:00,17544.0,17559.7,17538.0,17543.9,233.204
1438,2022-12-15 13:27:00,17538.0,17545.3,17532.3,17533.1,605.098


In [ ]:
df.where(df["volume"]>400)

,date,open,high,low,close,volume
0,2022-12-14 13:29:00,17928.5,17938.0,17901.9,17914.9,461.133
1,NaT,NaN,NaN,NaN,NaN,NaN
2,NaT,NaN,NaN,NaN,NaN,NaN
3,NaT,NaN,NaN,NaN,NaN,NaN
4,NaT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1435,NaT,NaN,NaN,NaN,NaN,NaN
1436,2022-12-15 13:25:00,17553.2,17563.1,17542.0,17542.0,520.463
1437,NaT,NaN,NaN,NaN,NaN,NaN
1438,2022-12-15 13:27:00,17538.0,17545.3,17532.3,17533.1,605.098


1671110978.1367543f"MACD {data}": 'MACD (0.00069874, 2.0162282023637734e-08, 8.819971600683938e-09)'


In [ ]:
# macding()
buying(AMOUNT,LONG,SHORT)
selling(AMOUNT,LONG,SHORT)
opening()
closing()
main().result()

1671110711.137985f"MACD {data}": 'MACD (0.00069896, -3.1224006429033356e-06, 2.1215652103046324e-06)'
1671110712.1376202f"MACD {data}": 'MACD (0.00069896, -3.0707090541441575e-06, 1.0831103574148742e-06)'
1671110713.1341832f"MACD {data}": 'MACD (0.00069896, -2.9952161107669713e-06, 2.674450637785051e-07)'
'13:25:13' - INFO - {'type': 'MARKET', 'symbol': 'DENTUSDT', 'quantity': 14970.0, 'positionSide': 'SHORT', 'side': 'SELL'}
1671110714.129198f"MACD {data}": 'MACD (0.00069895, -2.902733421673593e-06, -3.665906333119146e-07)'
1671110715.140417f"MACD {data}": 'MACD (0.0006989, -2.8011844952137866e-06, -8.53509405692289e-07)'
'13:25:16' - INFO - {'type': 'MARKET', 'symbol': 'DENTUSDT', 'quantity': 14970.0, 'positionSide': 'SHORT', 'side': 'SELL'}
1671110716.1385307f"MACD {data}": 'MACD (0.0006989, -2.6897010400648126e-06, -1.2207477325667937e-06)'
1671110717.1388693f"MACD {data}": 'MACD (0.00069892, -2.570109147397525e-06, -1.49062001553294e-06)'
'13:25:17' - INFO - {'type': 'MARKET', 'sy

KeyboardInterrupt: ignored

In [ ]:
!date

In [ ]:
income()

In [ ]:
income(False,False)

In [ ]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean() #properties of df
    exp2 = price.ewm(span = slow, adjust = False).mean()
    df = pd.DataFrame()
    df['price'] = price
    df['macd'] = exp1 - exp2
    df['signal'] = df['macd'].ewm(span = smooth, adjust = False).mean()
    df['hist'] = df['macd'] - df['signal']
    return df

cd = pd.DataFrame(CLOSES)
mac1 = get_macd(cd, 26, 12, 9)



In [ ]:
import multiprocessing,threading #standard in python 3

print(multiprocessing.cpu_count())
ic(threading.active_count())

In [ ]:
p = lambda x : print("SDSD")
def timeIT(fc,*args):
  k=[]
  for i in range(1000):
    x = time.time()
    y = time.time()
    fc(*args)
    k.append(x-y)
  z = np.array(k).mean()
  ic(fc.__name__,'took',z)
  return z

# timeIT(p)
cd = pd.DataFrame(CLOSES)
timeIT(get_macd,cd,26,12,9)
timeIT(talib.MACD,CLOSES,12,26,9) #Talib is faster apparently...

In [ ]:
c = CLOSES 
m,s,histogram = talib.MACD(c, fastperiod=12, slowperiod=26, signalperiod=9)
frames = talib.MACD(c, fastperiod=12, slowperiod=26, signalperiod=9)
# df = pd.concat([m,s,histogram], join = 'inner', axis = 1)
df = pd.DataFrame(frames).transpose()
df.columns = ['macd','signal','histogram']
# df.dropna(inplace=True)
df


# m,s = df.iloc[0,:2]

# net = m-s
# if m>]

# prev_net = net
# net = m-s
 

Ploting macd plus our Strategy

In [ ]:
def implement_macd_strategy(prices, data):    #for ploting
    buy_price = []
    sell_price = []
    macd_signal = []
    signal = 0

    for i in range(len(data)):
        if data['macd'][i] > data['signal'][i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        elif data['macd'][i] < data['signal'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            macd_signal.append(0)
            
    return buy_price, sell_price, macd_signal
            
buy_price, sell_price, macd_signal = MACDStrategy(df['price'], df)


In [ ]:
import matplotlib.pyplot as plt 


plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('fivethirtyeight')

def plot_macd(prices, macd, signal, hist):
    ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
    ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

    ax1.plot(prices)
    ax2.plot(macd, color = 'grey', linewidth = 1.5, label = 'MACD')
    ax2.plot(signal, color = 'skyblue', linewidth = 1.5, label = 'SIGNAL')

    for i in range(len(prices)):
        if hist[i]<0:
            ax2.bar(prices.index[i], hist[i], color = '#ef5350')
        else:
            ax2.bar(prices.index[i], hist[i], color = '#26a69a')

    plt.legend(loc = 'lower right')
df = mac1
l = [df[i].to_list() for i in df.columns][1:]
plot_macd(df['price'],*l)


In [ ]:
ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

ax1.plot(df['price'], color = 'skyblue', linewidth = 2, label = 'price')
ax1.plot(df.index, buy_price, marker = '^', color = 'green', markersize = 10, label = 'BUY SIGNAL', linewidth = 0)
ax1.plot(df.index, sell_price, marker = 'v', color = 'r', markersize = 10, label = 'SELL SIGNAL', linewidth = 0)
ax1.legend()
ax1.set_title('df MACD SIGNALS')
ax2.plot(df['macd'], color = 'grey', linewidth = 1.5, label = 'MACD')
ax2.plot(df['signal'], color = 'skyblue', linewidth = 1.5, label = 'SIGNAL')

for i in range(len(df)):
    if str(df['hist'][i])[0] == '-':
        ax2.bar(df.index[i], df['hist'][i], color = '#ef5350')
    else:
        ax2.bar(df.index[i], df['hist'][i], color = '#26a69a')
        
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
df['price'].tail()

In [ ]:
df

In [ ]:
positions()

In [ ]:
while 1:
  print(positions()['unRealizedProfit'].sum())
  time.sleep(5)


In [ ]:
# try:
#   loop = asyncio.get_running_loop()
#   main().result()
# except Exception as e:
#   lp = asyncio.get_event_loop()
#   # lp.stop()
#   print('Crash')
#   # loop.stop()`
#   print(e)

https://www.shanelynn.ie/using-python-threading-for-multiple-results-queue/
https://stackoverflow.com/questions/20056300/python-print-output-of-each-thread-to-seperate-file-no-processes

In [ ]:
#https://medium.com/codex/algorithmic-trading-with-macd-in-python-1c2769a6ad1b

In [ ]:
# bearish when macd <0
#bullish macd crosses >0


In [ ]:
#bearosj macd croses  bellow signal line

In [ ]:
# try:
#     loop = asyncio.get_running_loop()
# except RuntimeError:  # 'RuntimeError: There is no current event loop...'
#     loop = None

# if loop and loop.is_running():
#     print('Async event loop already running. Adding coroutine to the event loop.')
#     tsk = loop.create_task(main())
#     # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
#     # Optionally, a callback function can be executed when the coroutine completes
#     tsk.add_done_callback(
#         lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
# else:
#     print('Starting new event loop')
#     asyncio.run(main())

In [ ]:
#  miniticker_socket(self, update_time: int = 1000):
  # aggtrade_socket(
  #     symbol_ticker_socket(self, symbol: str):
  #     ticker_socket(self):
  #         def index_price_socket(self, symbol: str, fast: bool = True):


Prediction


In [ ]:
from fbprophet import Prophet


def predict(df,freq='1',period=60,mode='L',his=True):
  d = {'L':'linear','log':'logistic'}
  m = Prophet(growth=d.get(mode),daily_seasonality=True)

  m.fit(df)
  fut = m.make_future_dataframe(periods=period,freq=f'{freq}min',include_history=his)
  fore = m.predict(fut)
  fore['ds']  = fore['ds'].values.tolist()
  return fore

Historical 

In [ ]:
import numpy as np

df=history('BTC','1m','1 day ago')
# df['date']  = pd.to_datetime(df['date']).dt.time
make_history(df)

# make_history(df)

# df['ds'] = df['ds'].apply(lambda x: pd.Timestamp(x))
# df['ts'] = df.ds.values.astype(np.int64) 

In [ ]:
df['ds'] = df.ds.values.tolist()

In [ ]:
df.loc[df['ds']==1642402980000000000]

In [ ]:
ff = predict(df,1,100,his=False)
# ff['ds']=pd.to_datetime(df['ds']).dt.time
ff



In [ ]:
!date

In [ ]:
df[1440:]

In [ ]:
xf=history('EOS','1m','1 day ago')
make_history(xf)
predict(xf)

In [ ]:
x = df['ds'].iloc[1] -today()
x

In [ ]:
#Streaming

def listenKey(key=None,op=0):
  if op: return client.futures_stream_close(key)
  elif key==None:return client.futures_stream_get_listen_key()
  return client.futures_stream_keepalive(key)


In [ ]:
predict(df)

In [ ]:
predict(xf)

In [ ]:
m = Prophet(daily_seasonality=True)
m.fit(df)
fut = m.make_future_dataframe(periods=20,freq=f'{1}min',include_history=True)
fore = m.predict(fut)
fore.tail()

In [ ]:

m.plot(fore); #close->low 

In [ ]:
m.plot(fore); 

In [ ]:
m.plot(fore,figsize=(20,8));

In [ ]:
m.plot(fore,figsize=(20,8)); #linear

In [ ]:
m.plot(fore);

In [ ]:
m.plot_components(fore);

In [ ]:
fore[len(df):][['yhat','yhat_lower','yhat_upper','ds']]

https://towardsdatascience.com/a-quick-start-of-time-series-forecasting-with-a-practical-example-using-fb-prophet-31c4447a2274

https://stackoverflow.com/questions/59843459/forecasting-in-fb-prophet-by-minute


CandleSticks

Unarchives


In [ ]:
def snapshot(type='futures'): # Can be spot, margin,..
  type = type.upper()
  info = client.get_account_snapshot(type=type)
  return info

def assets():
  d = snapshot()
  asset = pd.DataFrame(d['snapshotVos'][0]['data']['position'])
  return d['snapshotVos'][4]['data']['assets']

def progress_balance():
   d = snapshot()['snapshotVos']
   assets = pd.DataFrame()
   for i in d:
     timestamp = '12' 
     k = pd.DataFrame(i['data']['assets'],index=[timestamp])
     assets = pd.concat([assets,k])
   return assets

# progress_balance()
# assets()

Account

In [ ]:
#Account

from binance.helpers import round_step_size

amount = 0.000234234
tick_size = 0.00001
rounded_amount = round_step_size(amount, tick_size)

orders = client.get_all_orders(symbol='BNBBTC', limit=10)


#Place Order
from binance.enums import *
order = client.create_order(
    symbol='BNBBTC',
    side=SIDE_BUY,
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_GTC,
    quantity=100,
    price='0.00001')

#Place Limit Order

order = client.order_limit_buy(
    symbol='BNBBTC',
    quantity=100,
    price='0.00001')

order = client.order_limit_sell(
    symbol='BNBBTC',
    quantity=100,
    price='0.00001')

#Market Order

order = client.order_market_buy(
    symbol='BNBBTC',
    quantity=100)

order = client.order_market_sell(
    symbol='BNBBTC',
    quantity=100)

#Test Order Creates and validates a new order but does not send it into the exchange.

from binance.enums import *
order = client.create_test_order(
    symbol='BNBBTC',
    side=SIDE_BUY,
    type=ORDER_TYPE_LIMIT,
    timeInForce=TIME_IN_FORCE_GTC,
    quantity=100,
    price='0.00001')

#Order Status
order = client.get_order(
    symbol='BNBBTC',
    orderId='orderId')

result = client.cancel_order(
    symbol='BNBBTC',
    orderId='orderId')

orders = client.get_open_orders(symbol='BNBBTC')

orders = client.get_all_orders(symbol='BNBBTC')

snapshot = client.get_asset_snapshot(asset='BTC')
status = client.get_account_status()
status = client.get_account_api_trading_status()
trades = client.get_my_trades(symbol='BNBBTC')
# get fees for all symbols
fees = client.get_trade_fee()

# get fee for one symbol
fees = client.get_trade_fee(symbol='BNBBTC')

details = client.get_asset_details()
log = client.get_dust_log()
transfer = client.transfer_dust(asset='BNZ')


Market Data Endpoints

In [ ]:
# Market Data Endpoints


depth = client.get_order_book(symbol='BNBBTC')
trades = client.get_recent_trades(symbol='BNBBTC')
trades = client.get_historical_trades(symbol='BNBBTC')
trades = client.get_aggregate_trades(symbol='BNBBTC')

agg_trades = client.aggregate_trade_iter(symbol='ETHBTC', start_str='30 minutes ago UTC')

# iterate over the trade iterator
for trade in agg_trades:
    print(trade)
    # do something with the trade data

# convert the iterator to a list
# note: generators can only be iterated over once so we need to call it again
agg_trades = client.aggregate_trade_iter(symbol='ETHBTC', '30 minutes ago UTC')
agg_trade_list = list(agg_trades)

# example using last_id value
agg_trades = client.aggregate_trade_iter(symbol='ETHBTC', last_id=23380478)
agg_trade_list = list(agg_trades)


candles = client.get_klines(symbol='BNBBTC', interval=Client.KLINE_INTERVAL_30MINUTE)
# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

for kline in client.get_historical_klines_generator("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC"):

avg_price = client.get_avg_price(symbol='BNBBTC')

tickers = client.get_ticker()
prices = client.get_all_tickers()
tickers = client.get_orderbook_tickers()#Get first bid and ask entry in the order book for all markets.



In [ ]:
import numpy as np
import talib
import time


flag = True

def RSIStrategy(price,period=14): #Long,Short Functions
    global PREV
    global TAKE
    global BALANCE
    global flag
    # global RSIS
    # RSI_Lock.acquire()
    if PREV!=-1:
      if price>PREV and TAKE==0: SDF[0]+=1
      elif price<PREV and TAKE==1: SDF[0]+=1
      elif PREV==price: SDF[1]+=1
      else:
        SDF[2]+=1 
    if CLOSES.size <=period: return 
    rsi = talib.RSI(CLOSES,period)
    cur_rsi = rsi[-1]

    try:
      if cur_rsi <= 30:
        print("OverSold Long")
        PREV = price    
        TAKE=0
        if SHORT.q>0:play(SHORT,price)
        if flag:
          LONG.open(AMOUNT,price)
          BALANCE-=p*_q
      elif cur_rsi > 70:
        print("OverBought Short")
        PREV = price
        TAKE=1
        if LONG.q>0:play(LONG,price)
        if flag:
          order(CRYPTO,'OPEN','SHORT',AMOUNT)#,trail=.5)
          BALANCE-=price*AMOUNT
          SHORT.open(AMOUNT,price)
    except Exception as e:
        print(e)
        print('hey',e.status_code,e.code)
        codes = ['-2027']
        if e.code ==-2027: #Mav leverage
          flag = False
          time.sleep(15)
        elif e.code==4045:
          cancel_order('BTC')

Async

In [ ]:
client = await AsyncClient.create(api_key, api_secret,testnet='true',tld='us',{"verify": False, "timeout": 20}) #testnet enables trading tId - region . Uses request package - so {} request params

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

    # fetch exchange info
    res = await client.get_exchange_info()
    print(json.dumps(res, indent=2))

  client.get_all_orders(symbol='BNBBTC', requests_params={'timeout': 5}) # override default params


In [ ]:
Simple API Endpoints
.ping()
time_res = client.get_server_time()
info = client.get_symbol_info('BNBBTC')
info = client.get_all_tickers() # Get information of coins (available for deposit and withdraw) for user



In [ ]:
%magic

Bibiliography

https://github.com/sammchardy/python-binance/blob/master/binance/client.py

https://colab.research.dfe.com/drive/1knvw5uUReCjrN7dbZNP-OzYs1OY5Mmc4#scrollTo=y_RiMszHFCZ_

https://algotrading101.com/learn/binance-python-api-guide/

https://medium.com/analytics-vidhya/futures-trading-with-python-binance-d738c71e17b5

https://binance-docs.github.io/apidocs/futures/en/#new-future-account-transfer

https://readthedocs.org/projects/python-binance/downloads/pdf/latest/


https://testnet.binancefuture.com/en/futures/BTCUSDT


